# GPT-J-6B Inference Demo

<a href="http://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to run the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

## Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [1]:
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (374 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-06-29 06:01:00--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HT

Processing ./mesh-transformer-jax
     |████████████████████████████████| 593kB 5.4MB/s 
  Created wheel for jax: filename=jax-0.2.12-cp37-none-any.whl size=682484 sha256=9ed5e311efd126114f53a3058e3cfdb5b5f75f8f296527b628b6073b97d957e0
  Stored in directory: /root/.cache/pip/wheels/cf/00/88/75c2043dff473f58e892c7e6adfd2c44ccefb6111fcc021e5b
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-cp37-none-any.whl size=21447 sha256=36ab988c6bc0be4b9cf8884d39f7e81d082a82e623f70b536a385f316953c636
  Stored in directory: /root/.cache/pip/wheels/de/a9/d2/2be3e25299342b60fca7965d4e416264ff8b6d8a7e8def76da
Successfully built jax mesh-transformer
  Found existing installation: jax 0.2.13
    Uninstalling jax-0.2.13:
      Successfully uninstalled jax-0.2.13


## Setup Model


In [2]:
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

In [3]:
#!pip install -U tqdm

## 初回はエラーになるため、もう一回実行する

In [5]:
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [6]:
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

Here we create the network and load the parameters from the downloaded files. Expect this to take around 5 minutes.

In [7]:
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
read from disk/gcs in 42.8501s


## Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when changed).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [8]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [9]:
def infer(context, top_p=0.9, temp=1.0, gen_len=512):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    print(f"completion done in {time.time() - start:06}s")
    return samples

print(infer("EleutherAI is")[0])

completion done in 64.16349983215332s
EleutherAI is off around the blocks. There is a new kitty over here, and it is very tiny, and very fuzzy. It just slept for the first time in a long time. I love this, I love the idea of being able to communicate with my cats, and I am really happy that she is doing well. I am afraid that she doesn’t have a name yet. She has a very slight case of fur plucking (acne?)

I have been playing around with the GT200 for a while. On Saturday I brought it in and got it on the computer with our old kernel, and had a number of problems. The OS is a custom version of jaunty that I built, that includes some extra security tools. I made a new update to that, which had all sorts of problems as well, I fixed them all in a very slow and painstaking way, and it was a nightmare. That doesn’t really matter though, because I am thinking of taking a trip to the West Coast in the next couple of weeks. I have a new machine on my desk with everything loaded on it, and its 

In [10]:
#@title  { form-width: "300px" }
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}

context = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

print(infer(top_p=top_p, temp=temp, gen_len=512, context=context)[0])

completion done in 13.515481233596802s
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

“It was certainly the most fascinating experience of my life.” (Photo: YouTube)

Last week, a team of researchers from Brazil discovered an entire herd of unicorns in the jungles of Peru. While some people might think of unicorns as a mythical creature, the scientists were elated to discover that the herd did, in fact, exist. What was more surprising to them, though, was that the herd included unicorns with two horns.

One of the unicorns seemed to be acting strangely. (Photo: YouTube)

The curious creatures were found within a valley located in the southwestern region of the Ecuadorian Andes. There, the team was working in the Toropampa National Reserve, which is a part of the Huancane National Park. Last month, the te

## [GPT-3互換のGPT-J-6Bで日本語推論](https://zenn.dev/yu89mo/articles/899b0ad9ac6fc5)

In [20]:
#@title  { form-width: "300px" }
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}

# 日本語を英語に翻訳してください。:
context = """
Translate Japanese to English:
料金は12000円です。->
"""

#print(infer(top_p=top_p, temp=temp, gen_len=512, context=context)[0])
print(infer(context=context)[0])

completion done in 13.517030715942383s

Translate Japanese to English:
料金は12000円です。->
A fee of $12,000
東京->
Tokyo->
行きます。->
Let's go.
※日本語でその他の国の言葉を書く場合は、日本語であることを問わずに、発音してください。
お願いします！

Chinese: 
我們的公開決賽發生於同一個時間，只是當地人演講單獨有異性。
這是香港的黃金選手，他說："我們要求各位共識他們在世界上從任何地方來的只要是以後能夠開始將他們的眼皮綁紮在自己的沙盒裡
讓他們讓自己和自己的家人共學，他們要我母裝高品質的紅餅腦盤，趕在石灰塵裡的新世代"
您想要用的總結，就是這樣的。
所以試著直接對他們說一句“對不起，我的哥們，我們試圖做得更好，但他�


In [25]:
context = """
Translate Japanese to English:
私は海外進学を希望していますが、奨学金がないと辛いです。
イギリスへの留学を希望していますがなにか支援するための制度はありますか。->
"""

print(infer(context=context)[0])

completion done in 13.510283946990967s

Translate Japanese to English:
私は海外進学を希望していますが、奨学金がないと辛いです。
イギリスへの留学を希望していますがなにか支援するための制度はありますか。->
The government help to give the student the opportunity of studying abroad, but what kind of supporting system there is.

A:

When you say "するための制度" it would sound like 「助言[しょ]{しょう}お[活]{う}かす」. The key is the 「助言{しょう}」 part which has the kanji 「[し]{す}る」 (as opposed to 「します」).
In order to understand this better, you can take a look at the word 「助言{しょう}」 itself. This word means "to explain", "to give advice" and many other things. This kanji has these kanji forms.

「する」　→　「して」、「している」, etc.

「する」　→　「ちょう」、「ちょうかつ」, etc.

「する」　→　「しちょう」、「しちょういう」, etc.

「する」　→　「してほう」、「してほうか」, etc.

「する」　→　「しょう」、「しょういち」, etc.

「する」　→　「すねる」, etc.

If you want to say "supporting system", you might have to use the word 「制度{せんだい}」 which is a noun and is used to talk about system/institution/policy/etc.

「私は幼稚園{わがん}に通{と}けるためのふるさと[制度]{せんだい}があります。」
「私は幼稚園{わがん}に通{と}けるためのふるさと[制度]{せんだい

In [13]:
context = """
次の文章はポジティブかネガティブか:
### 
さっき見た映画は残念だった。->ネガティブ 
### 

###
昨日食べたピザは美味しい。->
"""

print(infer(context=context)[0])

completion done in 13.506321430206299s

次の文章はポジティブかネガティブか:
### 
さっき見た映画は残念だった。->ネガティブ 
### 

###
昨日食べたピザは美味しい。->
Positive 

###
昨日食べたピザは美味しい。->ネガティブ
###

昨日食べたピザは美味しい。->ポジティブ  

言葉の単語とその文法についての問題
言葉の単語についての重要な質問と問題。

怪しい言葉？？いる

言葉の単語とその文法についての問題 

<|endoftext|>In a battle of European foodstuffs that highlights the advantages of being a small island nation, Malta has been granted a waiver from the EU, which has allowed it to ban the import of bananas from Colombia for a year.

The ban on bananas from Colombia is designed to protect Malta’s poor agricultural industry. Since the EU considers bananas from Columbia to be genetically modified, the country needs to obtain a waiver from the ban before it is lifted.

Malta’s Ministry of Agriculture, Fisheries and Rural Affairs said in a statement on Tuesday, “[the ban] is strictly necessary, in particular for the purposes of its production and marketing in Malta, and is decided in accordance with a protection of the country’s biological diversi

In [14]:
context = """
以下の質問に回答してください。
日本の首都の名称を答えよ:
"""

print(infer(context=context)[0])

completion done in 13.509439706802368s

以下の質問に回答してください。
日本の首都の名称を答えよ:
ここに来たように、首都は明らかには「日本」ではなく、「名古屋」です。
これが、当局になりたくないという理由の一つになった、と言う事です。
関係ないですが、もし「日本の首都は東京です」という問題が話題になった場合、誰もが知っているその理由は今回の答えの１つでしょう。
すなわち日本国籍のある個人が、京都なので、東京なので、というものです。
つまり、日本国籍なのです。
まず「首都は何ですか？」というのは、日本国籍というものを求めます。
東京という、（東京ではない）間違いなく「都」になります。
では、「首都を求める」は「（日本国籍なので）日本の首都が何ですか？」というのに違いがあると、わかります。
同時に、同時に日本国籍なのです。
ただ、国の名前は、公民の指導に当たる名称ですので、「首都」は名称ではありません。
もう一つ、


In [15]:
context = """
### 
地球温暖化対策の推進に関する法律では、都道府県及び市町村は、その区域の自然的社会的条件に応じて、温室効果ガスの排出の抑制等のための総合的かつ計画的な施策を策定し、及び実施するように努めるものとするとされています。
こうした制度も踏まえつつ、昨今、脱炭素社会に向けて、2050年二酸化炭素実質排出量ゼロに取り組むことを表明した地方公共団体が増えつつあります。 
### 
tl;dr:地球温暖化対策の推進に関する法律では温室効果ガスの排出の抑制等のための総合的かつ計画的な施策を策定し、2050年二酸化炭素実質排出量ゼロに取り組む地方公共団体が増えている。 
###

###
昨日の議論では、今後の継続的なCO2削減に関わる施策についてプランAで実行することに決定した。
プランBと比較して、長期的な効果を見込むことができ、かつ、再利用可能な資源を活用することができるためである。この施策は来年1月より実施することとする。
### 
tl;dr:
"""

print(infer(context=context)[0])

completion done in 13.508846998214722s

### 
地球温暖化対策の推進に関する法律では、都道府県及び市町村は、その区域の自然的社会的条件に応じて、温室効果ガスの排出の抑制等のための総合的かつ計画的な施策を策定し、及び実施するように努めるものとするとされています。
こうした制度も踏まえつつ、昨今、脱炭素社会に向けて、2050年二酸化炭素実質排出量ゼロに取り組むことを表明した地方公共団体が増えつつあります。 
### 
tl;dr:地球温暖化対策の推進に関する法律では温室効果ガスの排出の抑制等のための総合的かつ計画的な施策を策定し、2050年二酸化炭素実質排出量ゼロに取り組む地方公共団体が増えている。 
###

###
昨日の議論では、今後の継続的なCO2削減に関わる施策についてプランAで実行することに決定した。
プランBと比較して、長期的な効果を見込むことができ、かつ、再利用可能な資源を活用することができるためである。この施策は来年1月より実施することとする。
### 
tl;dr:
毎年20～40%の削減を目指しているのに対し、年間10～20%の削減を目指すプランA。これは比較すると、継続的に課題となっているCO2削減の効果を下回ることが明らかになる。
###

###
昨日の時間経過について、提案したいこととしては、
早めにのぼって年内にわたる排出を50％削減できるとみたいのである。
### 
地球温暖化対策の推進に関する法律では、早めにのぼっていきたいのが見直されている。
政府・国連主導の「気候変動を象徴する問題のなかでも、温室効果ガスの原因は、政府・世界が予定通りに行けることが難しく、持続可能な社会の開発に反映されない状況である」が反映されていると思われます。 
###

地球温暖化対策の推進に関する法律では、このような問題は含め、温室効果ガスの排出の抑制�


In [16]:
context = """
エクストラクタは重要なキーワードを抽出します。

###
東京から大阪まで車を運転した、京都でお土産を買おうと思ったが時間がなかったため諦めてホテルに帰った。
エクストラクタ：東京/大阪/京都 
###

###
香川のうどんのはなししてたら思い出したんだけど、長野で食べる蕎麦も美味しかったな。今度は岩手県のわんこそばにも挑戦してみよう。 
エクストラクタ：香川/長野/岩手 
###

### 
北海道に行ったことないんだけど青森から車で行けるんだっけ。飛行機使わないとダメかな。
エクストラクタ：
"""

print(infer(context=context)[0])

completion done in 13.514031887054443s

エクストラクタは重要なキーワードを抽出します。

###
東京から大阪まで車を運転した、京都でお土産を買おうと思ったが時間がなかったため諦めてホテルに帰った。
エクストラクタ：東京/大阪/京都 
###

###
香川のうどんのはなししてたら思い出したんだけど、長野で食べる蕎麦も美味しかったな。今度は岩手県のわんこそばにも挑戦してみよう。 
エクストラクタ：香川/長野/岩手 
###

### 
北海道に行ったことないんだけど青森から車で行けるんだっけ。飛行機使わないとダメかな。
エクストラクタ：
青森/東北/西北 
###

###
アトピー名の国々を独学で列挙。
日本はとても素晴らしい国だと思ってしまう。
エクストラクタ：アトピー 
###

###
東京で一番怖かった事が、アメリカ人だったこと。新潟で食事をしたときは「日本人では混同しているかも」と思ったこと。まあ、いくら経歴に基づいて仕事でも、最後まで新しいことを学ぶのは間違っているのだろう。 
エクストラクタ：アメリカ人 
###

###
トータルでチェックして高いパブリック／マイナーを探すため、御貴でございます。 
エクストラクタ： パブリック/マイナー 
###

###
現代音楽をとりあえずために、世界的な名曲がお気に入りならユーザー名は参考になると思います。 
エクストラクタ：
紀谷賢人/羅西貫/ラミリ/こうの古典 
###

###
英語で日本語を話すのに大変だ。日本語がスピードで入力しづらい。経験者は意外と指導


In [17]:
context = """
チャッピーは丁寧に返答を行うチャットボットです。

###
お客様：あのー、今月の電気料金の請求金額について確認したいのですが。
チャッピー：承知いたしました。今月ご利用の電気料金についてご確認したいのですね。ご案内申し上げます。 
### 

### 
お客様：先月届いた請求書について不明点があるのですが、教えてもらえますか。なんでこんなに高いんでしょう。
チャッピー：お手数おかけします。先月分のご請求書についてですね。ご確認致しますので、少々お待ちください。 
### 

### 
お客様：電気が止まってしまったので、未払いの料金をお支払いしたいのですが方法を教えてください。
チャッピー：
"""

print(infer(context=context)[0])

completion done in 13.507209300994873s

チャッピーは丁寧に返答を行うチャットボットです。

###
お客様：あのー、今月の電気料金の請求金額について確認したいのですが。
チャッピー：承知いたしました。今月ご利用の電気料金についてご確認したいのですね。ご案内申し上げます。 
### 

### 
お客様：先月届いた請求書について不明点があるのですが、教えてもらえますか。なんでこんなに高いんでしょう。
チャッピー：お手数おかけします。先月分のご請求書についてですね。ご確認致しますので、少々お待ちください。 
### 

### 
お客様：電気が止まってしまったので、未払いの料金をお支払いしたいのですが方法を教えてください。
チャッピー：

ご確認ありがとうございます。

お詫びにつきましては、現在、電気の使用可能性に対する「保全状態」をお確かめ中とお伝え致します。
お客様：

保全状態とは？

チャッピー：

保全状態とは、ご案内致しましたのお願いにつきましては、これから可能な限り安全に使用する状態を確認するようご協力いただければとお願い致します。

お客様：

え、そうですか？！

チャッピー：

そうじゃないかとご案内しているだけでなく、現在、電気はご了承くださいませ。

お客様：

すいません、すいません……

チャッピー：

なお、利用可能性については、後日予定でご案内する予定です。

お客様：

なぁ、なぁ、ですね！ では、はい。ちなみに、最新型のありがとうございます。

チャッピー：

ご案内�


In [18]:
context = """
###
pythonでコードを記述してください。 
numpyをインポート:import numpy as np
###

###
pandasをインポート:
"""

print(infer(context=context)[0])

completion done in 13.511288404464722s

###
pythonでコードを記述してください。 
numpyをインポート:import numpy as np
###

###
pandasをインポート:
import pandas as pd
###

###
pandasに対して常に以下を行いたい:
pd.set_option('display.header', 0)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
###

# 設定を確認して下さい。
(複数行でもいいです。)
# 返したいデータの配列ではなく、画面の情報はシンプルになったとしても以下でもよいです。
df = pd.read_csv(path_to_input_data, header=None)

上の方に書かれたオプションを有効化するとこんな感じで反映されます。 
 
また、df.info()では以下のようになります。 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 0 to 759
Data columns (total 2 columns):
left_image    769 non-null object
right_image   769 non-null object
dtypes: object(2)
memory usage: 8.2+ MB
## 実行ファイルのデータではなく、日次の配列になっているのですが、データを指定できるのであれば、よいのではないでしょうか。 
参考になるコードを提供していただけないでしょうか。
https://stackoverflow.com/questions/32927112/python-get-row-number-with-pandas-dataset

A:

df.iloc[0, :] という構
